In [5]:
import requests
company_name=input('Company name: ')
Company_profile=requests.get(f'https://financialmodelingprep.com/api/v3/search?query={company_name}&apikey=L7rFmF4kHeMh9H6f5pz5DehLND8DZ5Oz')
symbol=Company_profile.json()[0]['symbol']
print(symbol)



MSFT.NE


In [6]:
Balance_Sheet=requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}?period=annual&apikey=L7rFmF4kHeMh9H6f5pz5DehLND8DZ5Oz')
# leverage_financial_ratio Current ratio = Current assets / Current liabilities
if not Balance_Sheet.json():
    print('No Data')
else:
    current_assets=Balance_Sheet.json()[0]['totalCurrentAssets']
    current_liabilities=Balance_Sheet.json()[0]['totalCurrentLiabilities']
    current_ratio=current_assets/current_liabilities
    date=Balance_Sheet.json()[0]['date']
    print(date)
    print('Current Ratio: '+ str(current_ratio))
    if 1.2 <= current_ratio <= 2:
        print("A current ratio of", current_ratio, "is considered good. The business has enough current assets to cover its short-term liabilities.")
    elif current_ratio < 1:
        print("A current ratio of", current_ratio, "is below 1. This means that the company may not have enough liquid assets to cover its short-term liabilities.")
    else:
        print("The current ratio of", current_ratio, "is within an acceptable range, but it's not considered optimal.")
# Acid Test Ratio current assets - inventories/current liabilties "totalCurrentLiabilities"
inventory=Balance_Sheet.json()[0]['inventory']    
atr = (current_assets - inventory) / current_liabilities
print('Acid Test Ratio: ' + str(atr))
if atr > 1:
    print("An acid test ratio of", atr, "means that the company does have enough liquid assets to cover its short-term debts")
elif atr < 1:
    print("An acid test ratio of", atr, "means that the company does not have enough liquid assets to cover its short-term debts")
else:
    print("The acid test ratio", atr, "means that the company has just enough to cover its short-term debts")

# operatingCashFlow (opcf) = Operating cash flow / Current liabilities    
Cash_Flow=requests.get(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{symbol}?period=annual&apikey=L7rFmF4kHeMh9H6f5pz5DehLND8DZ5Oz')
opc=Cash_Flow.json()[0]["operatingCashFlow"]
opcf=opc/current_liabilities
print("Operating Cash Flow: "+ str(opcf))
                                                                                              

2023-06-30
Current Ratio: 1.76916725076573
A current ratio of 1.76916725076573 is considered good. The business has enough current assets to cover its short-term liabilities.
Acid Test Ratio: 1.745163179675273
An acid test ratio of 1.745163179675273 means that the company does have enough liquid assets to cover its short-term debts
Operating Cash Flow: 0.84092982169776


In [3]:
# Efficiency Ratios
# atr Asset turnover ratio = Net sales / Average total assets
